In [3]:
# %load ../../standard_import.txt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_seq_items', None)
 
#%config InlineBackend.figure_formats = {'pdf',}
%matplotlib inline

In [7]:
datos = pd.read_csv('online_shoppers_intention.csv')

datos_ok = datos.copy()
for i,val in enumerate(datos_ok.columns):
    labels = datos_ok[val].astype('category').cat.categories.tolist()
    dict_make = {val : {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}
    datos_ok[val].replace(dict_make[val], inplace=True)

X = np.array(datos_ok.drop(columns = 'Revenue').copy())
y = np.array(pd.factorize(datos_ok['Revenue'])[0])
print(X.shape)

(12330, 17)


In [ ]:
#hanwei

In [ ]:
#hanwei

In [ ]:
#hanwei

In [ ]:
#jhonny
import 

In [ ]:
#jhonny

In [ ]:
#jhonny